In [1]:
import sympy as sym
import sympy.physics.mechanics as me
from sympy.abc import t as time
import pandas as pd
import seaborn as sns

import numpy as np
from scipy.integrate import solve_ivp
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from scipy.optimize import fsolve,least_squares,root

import sys, os

sys.path.insert(1, os.path.join(sys.path[0], '../..'))
import custom_plot_objects as cpo

import ModelFramework as mf
import ModelFramework.Elements as ele
import ModelFramework.ExternalForces as ef
import FwtModels.RectWing as rw
import FwtModels.RollRig as RollRig

from matplotlib.lines import Line2D

me.mechanics_printing()

In [2]:
p = RollRig.base_params(1)

In [3]:
sm = mf.SymbolicModel.from_file('RollRigModel-Fixed.py')
sm.ExtForces = ef.CompositeForce([sm.ExtForces,ef.CustomForce(None)])
sm = sm.subs({p.alpha_r.name:0})

In [4]:
p.c.value = 0.0677
p.s.value = 1
p.sigma.value = 0.28
p.m_w.value = 0.22
p.m_f.value = 0.038
p.m_l.value = 0#0.0275
p.alpha_r.value = 0
p.c_d_max.value = 1
p.a_0.value = 2*np.pi
p.a_1.value = 2*np.pi
#p.tau.value = 0
p.alpha_r.value = 0
p.w_g.value = 0
p.beta.value = 0.0036
p.V.value = 22.5

#p.s.value = 0.72
#p.sigma.value = 0
#p.m_f.value = 0
#p.m_l.value = 0


p.Lambda.value = 0

In [5]:
nm = mf.NumericModel.from_SymbolicModel(p,sm)
ext_f = nm.ExtForces.force_funcs[0]

torque_period = 0.1
torque_max = p.beta.value*p.V.value**2

def torque(tup,x,t):
    if t<2:
        return 0
    elif t<torque_period/2+2:
        return torque_max*0.5*(1- np.cos(2*np.pi*(t-2)/torque_period))
    else:
        return torque_max
    
def custom_force(tup,x,t):
    return np.array([[torque(tup,x,t)]])

nm.ExtForces.force_funcs = [ext_f , custom_force]



In [6]:
y = [1.5,0]
nm.deriv(0,y,p.GetNumericTuple(y,0))

In [7]:
def GenRunData(model,p,ic,end_time,xNames=None,additional_cols = {},sample_freq=100):
    
    if xNames==None:
        xNames = [f'x{i}' for i in range(p.qs*2)]
    
    # create objective function and solve
    def objectiveDeriv(t,y):
        return nm.deriv(t,y,p.GetNumericTuple(y,t))
    yData = solve_ivp(objectiveDeriv,(0,end_time),ic,max_step=1/sample_freq)
    
    t = np.linspace(0,end_time,(end_time*100)+1)
   
    #interpolate to reduce t and y points
    int_func = interp1d(yData.t,yData.y)  
    yi = int_func(t)
    
    #generate list
    tau = [torque((0,),[0],i) for i in t]
    q_data = [{names[i]:v for i,v in enumerate(row)}for row in yi.T]    
    params = {k.name:v for k,v in p.GetSubs(0,None).items()}


    data = [{'t':_t,'torque':tau[i],'Locked':True,**q_data[i],**params,**additional_cols} for i,_t in enumerate(t)]
    return data
    

In [8]:
p.Lambda.value

In [9]:
names = ['Roll','Roll Rate','Left FWT Angle','Left FWT Velocity','Right FWT Angle','Right FWT Velocity']

res = []

for mode in ['Locked','Removed']:
    if mode =='Locked':
        p.m_f.value = 0.038
        p.a_1.value = 2*np.pi
    else:
        p.m_f.value = 0
        p.a_1.value = 0
        p.s.value = 1
    for v in np.linspace(15,30,4):
        p.V.value = v
        ext_f = nm.ExtForces.force_funcs[0]

        torque_period = 0.1       
        for tm in [3,2,1]:
            p.beta.value = 0.0036*tm/3*0.7
            torque_max = p.beta.value*p.V.value**2

            def torque(tup,x,t):
                if t<2:
                    return 0
                elif t<torque_period/2+2:
                    return torque_max*0.5*(1- np.cos(2*np.pi*(t-2)/torque_period))
                else:
                    return torque_max

            def custom_force(tup,x,t):
                return np.array([[torque(tup,x,t)]])

            nm.ExtForces.force_funcs = [ext_f , custom_force]

            res+=GenRunData(nm,p,[0,0],10,names[:2],{'Mode':mode,'AileronAngle':(4-tm)*7,**{n:0 for n in names[2:]}},100)
df = pd.DataFrame(res)
df.to_pickle('FixedData.pkl')

<string>:49: RuntimeWarning: divide by zero encountered in double_scalars
<string>:50: RuntimeWarning: invalid value encountered in double_scalars
<string>:54: RuntimeWarning: divide by zero encountered in double_scalars
<string>:63: RuntimeWarning: invalid value encountered in double_scalars
<string>:64: RuntimeWarning: invalid value encountered in double_scalars


ValueError: x and y arrays must have at least 2 entries

In [49]:
df['Roll Rate [Deg]']=np.rad2deg(df['Roll Rate'])
df[df['t']>4].groupby(['Mode','V'])['Roll Rate [Deg]'].mean()

Mode     V   
Locked   15.0     66.540488
         20.0     88.720651
         25.0    110.900814
         30.0    133.080977
Removed  15.0    177.589806
         20.0    236.786408
         25.0    295.983011
         30.0    355.179613
Name: Roll Rate [Deg], dtype: float64

In [50]:
df.groupby(['Mode','V','torque'])['Left FWT Angle'].mean()

Mode     V     torque  
Locked   15.0  0.000000    0
               0.018048    0
               0.036096    0
               0.054144    0
               0.065298    0
                          ..
Removed  30.0  1.367617    0
               1.484425    0
               1.512000    0
               2.051425    0
               2.268000    0
Name: Left FWT Angle, Length: 128, dtype: int64

In [51]:
df['Lambda']

0        0
1        0
2        0
3        0
4        0
        ..
24019    0
24020    0
24021    0
24022    0
24023    0
Name: Lambda, Length: 24024, dtype: int64

In [52]:
df.to_csv(r'/Users/fintan/Git/FlexiFWT_DAQ/Analysis/RollingRigV3/ModelComparison/FixedData.csv', index = False)